In [1]:
from allennlp.modules.elmo import Elmo, batch_to_ids

options_file = "./small_emlo/elmo_2x1024_128_2048cnn_1xhighway_options.json"
weight_file = "./small_emlo/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5"

In [2]:
#  use batch_to_ids to convert sentences to character ids
sentences = [['First', 'sentence', '.'], ['Another', '.']]
character_ids = batch_to_ids(sentences)
print(character_ids.size())
character_ids
# First : <s>, F, i, r, s, t, </s>
# sentence : <s>, s, e, n, t, e, n, c, e, </s>
# <s> 259; </s> 260

torch.Size([2, 3, 50])


tensor([[[259,  71, 106, 115, 116, 117, 260, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259, 116, 102, 111, 117, 102, 111, 100, 102, 260, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261],
         [259,  47, 260, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 261]],

        [[259,  66, 111, 112, 117, 105, 102, 115, 260, 261, 261, 261, 261, 261,
          261, 261, 261, 261, 261, 261, 261, 

In [11]:
elmo = Elmo(options_file, weight_file, 1, dropout=0)
embeddings = elmo(character_ids)
embeddings

01/16/2019 10:17:05 - INFO - allennlp.modules.elmo -   Initializing ELMo


{'elmo_representations': [tensor([[[-0.1290, -0.0053, -0.5269,  ...,  0.1373,  0.2507, -0.7911],
           [ 0.5511, -0.2235,  0.1626,  ..., -0.8953,  0.5011, -1.4219],
           [ 0.3206,  0.2114,  0.2246,  ..., -0.2516, -0.1505,  0.2238]],
  
          [[ 0.6634,  0.5339, -0.4762,  ...,  0.2827,  0.2224,  0.0898],
           [ 0.0397,  0.2241,  0.1264,  ..., -0.2516, -0.1505,  0.2238],
           [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]]],
         grad_fn=<CopySlices>)], 'mask': tensor([[1, 1, 1],
         [1, 1, 0]])}

In [12]:
embeddings['elmo_representations'][0].size() # batch_size x seq_len x embed_dim

torch.Size([2, 3, 256])

In [13]:
embeddings['elmo_representations'][0].requires_grad

True

In [3]:
elmo1 = Elmo(options_file, weight_file, 1, dropout=0, requires_grad=False)
embeddings1 = elmo1(character_ids)
assert embeddings1['elmo_representations'][0].requires_grad == True
print([x.requires_grad for x in elmo1._elmo_lstm.parameters()])
all(x.requires_grad == False for x in elmo1._elmo_lstm.parameters())

01/16/2019 10:35:05 - INFO - allennlp.modules.elmo -   Initializing ELMo


[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]


True

In [4]:
print([x.requires_grad for x in elmo1.scalar_mix_0.parameters()])

[True, True, True, True]


## Explaination before

This is expected. Within the Elmo module, you need a trainable scalar weight matrix that produces a mixtures of the different elmo layers. an operation with a requires_grad=False tensor and a requires_grad=True tensor (the mixture weights) produces the resultant requires_grad=True tensor that you see.

The elmo requires_grad construction param should actually be called trainable --- it affects whether the LSTM parameters within the Elmo module get updated during training or not. By passing requires_grad=False into the constructor, we can verify that the LSTM parameters are frozen with:


In [ ]:
# from allennlp.modules.elmo import Elmo, batch_to_ids
# options_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_options.json"
# weight_file = "https://s3-us-west-2.amazonaws.com/allennlp/models/elmo/2x4096_512_2048cnn_2xhighway/elmo_2x4096_512_2048cnn_2xhighway_weights.hdf5"

# elmo = Elmo(options_file, weight_file, 1, dropout=0, requires_grad=False)
# all(x.requires_grad == False for x in elmo._elmo_lstm.parameters())
# >>>True

In [1]:
import torch
from elmo_newspaper import ELMoGRU


options_file = "./small_emlo/elmo_2x1024_128_2048cnn_1xhighway_options.json"
weight_file = "./small_emlo/elmo_2x1024_128_2048cnn_1xhighway_weights.hdf5"
    
device = torch.device("cuda:1" if torch.cuda.is_available() else "cpu")
model = ELMoGRU(device, options_file, weight_file, batch_size=8)
model.load_state_dict(torch.load("elmo_newspaper_0115.pt"))
model

01/16/2019 09:59:01 - INFO - allennlp.modules.elmo -   Initializing ELMo


ELMoGRU(
  (elmo): Elmo(
    (_elmo_lstm): _ElmoBiLm(
      (_token_embedder): _ElmoCharacterEncoder(
        (char_conv_0): Conv1d(16, 32, kernel_size=(1,), stride=(1,))
        (char_conv_1): Conv1d(16, 32, kernel_size=(2,), stride=(1,))
        (char_conv_2): Conv1d(16, 64, kernel_size=(3,), stride=(1,))
        (char_conv_3): Conv1d(16, 128, kernel_size=(4,), stride=(1,))
        (char_conv_4): Conv1d(16, 256, kernel_size=(5,), stride=(1,))
        (char_conv_5): Conv1d(16, 512, kernel_size=(6,), stride=(1,))
        (char_conv_6): Conv1d(16, 1024, kernel_size=(7,), stride=(1,))
        (_highways): Highway(
          (_layers): ModuleList(
            (0): Linear(in_features=2048, out_features=4096, bias=True)
          )
        )
        (_projection): Linear(in_features=2048, out_features=128, bias=True)
      )
      (_elmo_lstm): ElmoLstm(
        (forward_layer_0): LstmCellWithProjection(
          (input_linearity): Linear(in_features=128, out_features=4096, bias=False)
   

In [3]:
for param_tensor in model.state_dict():
    print(param_tensor, '\t', model.state_dict()[param_tensor].size())

elmo._elmo_lstm._token_embedder._char_embedding_weights 	 torch.Size([262, 16])
elmo._elmo_lstm._token_embedder.char_conv_0.weight 	 torch.Size([32, 16, 1])
elmo._elmo_lstm._token_embedder.char_conv_0.bias 	 torch.Size([32])
elmo._elmo_lstm._token_embedder.char_conv_1.weight 	 torch.Size([32, 16, 2])
elmo._elmo_lstm._token_embedder.char_conv_1.bias 	 torch.Size([32])
elmo._elmo_lstm._token_embedder.char_conv_2.weight 	 torch.Size([64, 16, 3])
elmo._elmo_lstm._token_embedder.char_conv_2.bias 	 torch.Size([64])
elmo._elmo_lstm._token_embedder.char_conv_3.weight 	 torch.Size([128, 16, 4])
elmo._elmo_lstm._token_embedder.char_conv_3.bias 	 torch.Size([128])
elmo._elmo_lstm._token_embedder.char_conv_4.weight 	 torch.Size([256, 16, 5])
elmo._elmo_lstm._token_embedder.char_conv_4.bias 	 torch.Size([256])
elmo._elmo_lstm._token_embedder.char_conv_5.weight 	 torch.Size([512, 16, 6])
elmo._elmo_lstm._token_embedder.char_conv_5.bias 	 torch.Size([512])
elmo._elmo_lstm._token_embedder.char_conv_6.

In [8]:
model.state_dict()['fc.weight'].requires_grad

False